In [1]:
#change current path to "../../"
import os
import sys
sys.path.append("../../")

In [2]:
from Model.Environment import Game, GameObserver, Environment, TransitionProfile
import ipywidgets as widgets
from IPython.display import display

In [3]:
game = Game(3, [2, 3, 3])

In [4]:
game.setPossibleActions([3, 3, 3])

In [5]:
import numpy as np

In [6]:
a = np.ones((2, 3, 3))
a = np.pad(a, ((0, 1), (0, 0), (0, 0)), mode='constant', constant_values=0)
a.shape

(3, 3, 3)

In [7]:
env = Environment(2, 2)

In [8]:
print(game)

Game: [3, 3, 3] 
 PossibileActions:[3, 3, 3] 
 NPlayers: 3 
 Transition: ['{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}'] 
 Payoff: [[[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]


 [[[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]

  [[0. 0. 0.]
   [0. 0. 0.]
   [0. 0. 0.]]]]


In [9]:
game.setNPlayers(2)
print(game)
game.setNPlayers(3)
print(game)
game.setTransition((0, 0, 0), 1, 0.5)

Game: [3, 3] 
 PossibileActions:[3, 3] 
 NPlayers: 2 
 Transition: ['{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}'] 
 Payoff: [[[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]

 [[0. 0.]
  [0. 0.]
  [0. 0.]]]
Game: [3, 3, 1] 
 PossibileActions:[3, 3, 1] 
 NPlayers: 3 
 Transition: ['{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}', '{}'] 
 Payoff: [[[[0. 0. 0.]]

  [[0. 0. 0.]]

  [[0. 0. 0.]]]


 [[[0. 0. 0.]]

  [[0. 0. 0.]]

  [[0. 0. 0.]]]


 [[[0. 0. 0.]]

  [[0. 0. 0.]]

  [[0. 0. 0.]]]]


In [10]:
nPlayersWidget = widgets.IntSlider(
    value=game.NPlayers,
    min=2,
    max=4,
    step=1,
    description='Number of players:',
)

In [11]:
nPlayersWidget.observe(lambda change: game.setNPlayers(change.new), names='value')

In [12]:
display(nPlayersWidget)

IntSlider(value=3, description='Number of players:', max=4, min=2)

In [13]:
class ActionDomainsWidgets(GameObserver):
    def __init__(self, game):
        self.game = game
        self.widgets = []

        self.box = widgets.GridBox(layout=widgets.Layout(grid_template_columns="repeat(2, 50%)"))

        self.title = widgets.HTML(value="<h2>Action domains</h2>")

        self.widget = widgets.VBox([self.title, self.box])

        self.update(game)

        game.attach(self)
        
    def update(self, game):
        if(game.NPlayers < len(self.widgets)):
            self.box.children = self.box.children[:game.NPlayers]
            self.widgets = self.widgets[:game.NPlayers]
        else:
            for i in range(min(game.NPlayers, len(self.widgets))):
                self.widgets[i].value = game.possibleActions[i]

            for i in range(max(0, game.NPlayers - len(self.widgets))):
                newWidget = widgets.IntSlider(
                    value=game.possibleActions[len(self.widgets)],
                    min=1,
                    max=10,
                    step=1,
                    description='Player '+str(len(self.widgets))+':',
                )
                self.widgets.append(newWidget)
                newWidget.observe(lambda change : self.setPossibleActions(), names='value')

            self.box.children = self.widgets

    def getWidget(self):
        return self.widget

    def setPossibleActions(self):
        actions = [w.value for w in self.widgets]
        self.game.setPossibleActions(actions)

In [14]:
actionDomainsWidgets = ActionDomainsWidgets(game)
display(actionDomainsWidgets.getWidget())

In [15]:
class ActionProfileWidget(GameObserver):
    def __init__(self, game, on_change_callbacks=[]):
        self.game = game
        self.box = widgets.GridBox(layout=widgets.Layout(grid_template_columns="repeat(2, 50%)"))
        self.widgets = []

        self.on_change_callbacks = on_change_callbacks

        self.title = widgets.HTML(value="<h2>Action Profile Selection</h2>")

        self.widget = widgets.VBox([self.title, self.box])

        game.attach(self)

        self.update(game)
            
        
    def getWidget(self):
        return self.widget
    
    def get(self):
        return tuple([w.value for w in self.widgets])
    
    def update(self, game):
        if(game.NPlayers < len(self.widgets)):
            self.box.children = self.box.children[:game.NPlayers]
            self.widgets = self.widgets[:game.NPlayers]
        else:
            for i in range(min(game.NPlayers, len(self.widgets))):
                self.widgets[i].max = game.possibleActions[i]-1

            for i in range(max(0, game.NPlayers - len(self.widgets))):
                newWidget = widgets.IntSlider(
                    value=0,
                    min=0,
                    max=game.possibleActions[len(self.widgets)]-1,
                    step=1,
                    description='Player '+str(len(self.widgets))+':',
                )
                self.widgets.append(newWidget)
                newWidget.observe(self.onChange, names='value')

            self.box.children = self.widgets

    def setOnChanges(self):
        for w in self.widgets:
            w.observe(self.onChange, names='value')

    def addOnChangeCallback(self, callback):
        self.on_change_callbacks.append(callback)
        for w in self.widgets:
            w.observe(callback, names='value')

    def onChange(self, change):
        for callback in self.on_change_callbacks:
            callback(change)
        


In [16]:
display(nPlayersWidget)

IntSlider(value=3, description='Number of players:', max=4, min=2)

In [17]:
actionProfileWidget = ActionProfileWidget(game, [lambda x : print(actionProfileWidget.get())])

In [18]:
class TransitionProbabilityWidget(GameObserver):
    def __init__(self, game, NGames = 2, actionProfile = (0, 0)):
        self.game = game
        self.NGames = NGames
        self.box = widgets.GridBox(layout=widgets.Layout(grid_template_columns="repeat(2, 50%)"))
        self.widgets = []
        self.actionProfile = actionProfile

        self.normalizeButton = widgets.Button(
            description='Normalize',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
            tooltip='Normalize',
            icon='check' # (FontAwesome names without the `fa-` prefix)
        )

        self.normalizeButton.on_click(lambda x : self.normalizeProbabilities())

        self.title = widgets.HTML()
        self.updateTitle()

        self.widget = widgets.VBox([self.title, self.box, self.normalizeButton])

        game.attach(self)

        self.update(game)
                    
    def getWidget(self):
        return self.widget
    
    def get(self):
        return tuple([w.value for w in self.widgets])
    
    def update(self, game):
        if(self.NGames < len(self.widgets)):
            self.box.children = self.box.children[:game.NGames]
            self.widgets = self.widgets[:game.NGames]
        else:
            nexts, ps = game.getTransition(self.actionProfile).getTransitions()

            for i in range(max(0, self.NGames - len(self.widgets))):
                val = 0
                if(len(self.widgets) in nexts):
                    val = game.getTransition(self.actionProfile).getProbability(len(self.widgets))

                newWidget = widgets.BoundedFloatText(
                    value=val,
                    min=0,
                    max=1,
                    step=0.01,
                    description='Game '+str(len(self.widgets))+':',
                )
                self.widgets.append(newWidget)
                newWidget.observe(lambda change : 
                                    self.setTransitionProbs(),
                                    names='value')
                
            for i in range(min(self.NGames, len(self.widgets))):
                if i not in nexts:
                    self.widgets[i].value = 0

            for n, p  in zip(nexts, ps):
                self.widgets[n].value = p

            self.box.children = self.widgets

    def normalizeProbabilities(self):
        total = sum([w.value for w in self.widgets])

        if total == 0:
            return
        
        for w in self.widgets:
            w.value /= total

    def setTransitionProbs(self):
        tp = TransitionProfile({})

        for i, w in enumerate(self.widgets):
            if(w.value > 0):
                tp.setTransition(i, w.value)

        self.game.setTransitionProfile(self.actionProfile, tp)

    def updateTitle(self):
        self.title.value = "<h2>Transition probabilities - "+str(self.actionProfile)+"</h2>"


    def updateActionProfile(self, actionProfile):
        self.actionProfile = actionProfile
        transitionProfile = game.getTransition(tuple(actionProfile))
        nextGames, porobabilities = transitionProfile.getTransitions()

        for (i, w) in enumerate(self.widgets):
            if(i not in nextGames):
                w.value = 0
        
        for g, p in zip(nextGames, porobabilities):
            self.widgets[g].value = p

        self.updateTitle()


In [19]:
transitionProbabilityWidget = TransitionProbabilityWidget(game, 2, actionProfileWidget.get())

In [20]:
actionProfileWidget.addOnChangeCallback(lambda x : transitionProbabilityWidget.updateActionProfile(actionProfileWidget.get()))

In [21]:
display(actionProfileWidget.getWidget())

In [22]:
class PayoffWidget(GameObserver):
    def __init__(self, game, actionProfile = (0, 0, 0)):
        self.game = game
        self.box = widgets.GridBox(layout=widgets.Layout(grid_template_columns="repeat(2, 50%)"))
        self.widgets = []
        self.actionProfile = actionProfile

        self.title = widgets.HTML()
        self.updateTitle()

        self.widget = widgets.VBox([self.title, self.box])

        game.attach(self)

        self.update(game)
                    
    def getWidget(self):
        return self.widget
    
    def get(self):
        return tuple([w.value for w in self.widgets])
    
    def update(self, game):
        payoffs = game.getPayoff(self.actionProfile)

        if(len(self.widgets) > game.NPlayers):
            self.widgets = self.widgets[:game.NPlayers]
            self.box.children = self.box.children[:game.NPlayers]

            for i in range(len(self.widgets)):
                self.widgets[i].value = payoffs[i]

        else:
            for i in range(len(self.widgets)):
                self.widgets[i].value = payoffs[i]

            for i in range(0 , game.NPlayers - len(self.widgets)):
                newWidget = widgets.FloatText(
                    value=0,
                    min=0,
                    max=1,
                    step=0.01,
                    description='Player '+str(len(self.widgets))+':',
                )
                self.widgets.append(newWidget)
                newWidget.observe(lambda change : 
                                    self.setPayoff(),
                                    names='value')
            self.box.children = self.widgets

    def setPayoff(self):
        payoffs = [w.value for w in self.widgets]

        self.game.setPayoff(self.actionProfile, payoffs)

    def updateActionProfile(self, actionProfile):
        self.actionProfile = actionProfile
        self.updateTitle()
        self.update(self.game)

    def updateTitle(self):
        self.title.value = "<h2>Payoffs - "+str(self.actionProfile)+"</h2>"

    def getWidget(self):
        return self.widget

In [23]:
payoffWidget = PayoffWidget(game, actionProfileWidget.get())

In [24]:
actionProfileWidget.addOnChangeCallback(lambda x : payoffWidget.updateActionProfile(actionProfileWidget.get()))

In [25]:
display(actionProfileWidget.getWidget())

In [26]:
class GameEditor:
    nPlayers = 1
    def __init__(self, game, NGames = 2, actionProfile = (0, 0)):
        self.game = game
        self.nPlayers = game.NPlayers

        self.title = widgets.HTML(value="<h1>Game Editor</h1>")

        self.actionDomainsWidgets = ActionDomainsWidgets(game)

        self.actionProfileWidget = ActionProfileWidget(game, [])

        self.TransitionProbabilityWidget = TransitionProbabilityWidget(game, 2, actionProfileWidget.get())
        self.PayoffWidget = PayoffWidget(game, actionProfileWidget.get())

        self.actionProfileWidget.addOnChangeCallback(lambda x : self.TransitionProbabilityWidget.updateActionProfile(self.actionProfileWidget.get()))
        self.actionProfileWidget.addOnChangeCallback(lambda x : self.PayoffWidget.updateActionProfile(self.actionProfileWidget.get()))

        self.box = widgets.VBox([
            self.title,
            self.actionDomainsWidgets.getWidget(),
            self.actionProfileWidget.getWidget(),
            self.TransitionProbabilityWidget.getWidget(),
            self.PayoffWidget.getWidget()
        ])

    def getWidget(self):
        return self.box

In [27]:
gameEditor = GameEditor(game)
display(gameEditor.getWidget())

In [28]:
class EnvironmentWidget:
    def __init__(self, env) -> None:
        self.env = env

        self.widgets = [
            GameEditor(game) for game in env.getGames()
        ]

        self.box = widgets.Tab(
            children=[w.getWidget() for w in self.widgets],
        )

        self.box.titles = ["Game " + str(i) for i in range(len(self.widgets))]

In [29]:
env2 = Environment(2, 3)

In [30]:
env = Environment(NGames=2, NPlayers=3)
environmentWidget = EnvironmentWidget(env)
display(environmentWidget.box)